In [1]:
import pandas as pd
import torch
import pickle
import time
from sklearn.metrics import classification_report
from sklearn.preprocessing import MinMaxScaler
from mlp import MLP
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
from torch import nn
import torch
import torchvision
from torch import nn, optim
import torch.nn.functional as F
from torchvision import datasets, transforms

- Note on Scaling below: 
        - The test set for SVM has been scaled based on the original training set.
        - For MLP a validation set was split from the training set and so the test and validation sets were scaled on that               smaller training set to prevent data leakage. 
        - This is why there are two different scalings for SVM and MLP below, the test set is the same in both.

# Import Data and Scale Test Set - SVM:

In [2]:
X_test = pd.read_csv('Data/X_test.csv')
y_test = pd.read_csv('Data/y_test.csv', header = None)
X_train = pd.read_csv('Data/X_train.csv')
y_train = pd.read_csv('Data/y_train.csv', header=None)

In [3]:
print("X_train Length: {:,}".format(len(X_train)))
print("X_test Length:  {:,}".format(len(X_test)))
print("~~~~~~~~~~~~~~~~~~~~~~~~")
print("y_train Length: {:,}".format(len(y_train)))
print("y_test Length:  {:,}".format(len(y_test)))

X_train Length: 464,809
X_test Length:  116,203
~~~~~~~~~~~~~~~~~~~~~~~~
y_train Length: 464,809
y_test Length:  116,203


In [3]:
# Definining Scaler:
scaler = MinMaxScaler()
# Fitting scaler on X_train:
X_train = scaler.fit_transform(X_train)
# Applying to test set:
X_test = scaler.transform(X_test)

# Functions to Calculate Overall Accuracy:

In [4]:
# Used to calculate overall accuracy:
def calc_acc_svm(y_pred, y_test):
    
    # Calculates correct predictions by comparing to actual values
    correct_pred = (y_pred == y_test)
    accuracy = correct_pred.sum() / len(correct_pred)
    
    accuracy = accuracy * 100
    
    return accuracy

In [5]:
# Defining a function to calculate accuracy of predictions intra-batch:

def calc_acc_mlp(y_pred, y_test):
    # Applies a softmax followed by a log to the y_pred tensor:
    y_pred_torch = torch.log_softmax(y_pred, dim = 1)
    
    # Returns the predicted values for each batch datapoint
    _, y_pred_values = torch.max(y_pred_torch, dim = 1)    
    
    # Calculates correct predictions by comparing to actual values
    correct_pred = (y_pred_values == y_test).float()
    accuracy = correct_pred.sum() / len(correct_pred)
    
    accuracy = accuracy * 100
    
    return accuracy

# Testing the best SVM Model:

In [ ]:
# To load the model:
svm_best_model = pickle.load(open('Models/svm_basic_model.sav', 'rb'))

In [7]:
# Used to predict the test set, time included as an indication of how long it will take (seconds):
start = time.time()
y_pred_SVM = svm_best_model.predict(X_test)
print(time.time() - start)

2084.9346647262573


In [24]:
# Overall Accuracy of the Model:
calc_acc_svm(y_pred_SVM, y_test[0])

86.51755978761305

In [20]:
# Printing key metrics:
print(classification_report(y_test, y_pred_SVM, labels=[1, 2, 3, 4, 5, 6, 7]))

              precision    recall  f1-score   support

           1       0.88      0.82      0.85     42368
           2       0.86      0.91      0.88     56661
           3       0.87      0.89      0.88      7151
           4       0.88      0.80      0.84       549
           5       0.84      0.54      0.66      1899
           6       0.80      0.73      0.76      3473
           7       0.93      0.90      0.91      4102

    accuracy                           0.87    116203
   macro avg       0.87      0.80      0.83    116203
weighted avg       0.87      0.87      0.86    116203



# Testing the best MLP Model:

## Loading and Scaling Data:

In [9]:
X_train_mlp = pd.read_csv('Data/X_train_mlp.csv')
X_val_mlp = pd.read_csv('Data/X_val_mlp.csv')
X_test_mlp = pd.read_csv('Data/X_test.csv')
y_train_mlp = pd.read_csv('Data/y_train_mlp.csv', header = None)
y_val_mlp = pd.read_csv('Data/y_val_mlp.csv', header = None)
y_test_mlp = pd.read_csv('Data/y_test.csv', header = None)

In [18]:
print("X_train Length: {:,}".format(len(X_train_mlp)))
print("X_val Length:   {:,}".format(len(X_val_mlp)))
print("X_test Length:  {:,}".format(len(X_test_mlp)))
print("~~~~~~~~~~~~~~~~~~~~~~~~")
print("y_train Length: {:,}".format(len(y_train_mlp)))
print("y_val Length:   {:,}".format(len(y_val_mlp)))
print("y_test Length:  {:,}".format(len(y_test_mlp)))

X_train Length: 371,847
X_val Length:   92,962
X_test Length:  116,203
~~~~~~~~~~~~~~~~~~~~~~~~
y_train Length: 371,847
y_val Length:   92,962
y_test Length:  116,203


In [10]:
# Definining Scaler:
scaler = MinMaxScaler()

# Fitting scaler on X_train:
X_train_mlp = scaler.fit_transform(X_train_mlp)

# Applying this scaling to the test set:
X_test_mlp = scaler.transform(X_test_mlp)

In [11]:
# Converting to tensors:
X_test_tensor = torch.from_numpy(X_test_mlp).float()
y_test_tensor = torch.from_numpy(y_test_mlp.values).long() - 1

## Creating Dataloader Object to load the test set in batches:

In [12]:
# Creating a dataset class based on torch.utils.data.Dataset that can be loaded using train_loader
# source: https://pytorch.org/tutorials/beginner/data_loading_tutorial.html

class CreatePytorchDataset(Dataset):
    
    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data
    
    # Altering __len__ and __getitem__ methods from Dataset, so len returns size of dataset and get item 
    # allows dataset indexing by converting any tensor to a list
    
    def __len__(self):
        return len(self.X_data)
    
    def __getitem__(self, idx):
        return self.X_data[idx], self.y_data[idx]

In [13]:
# Creating Pytorch Dataset using the imported function above:
test_dataset = CreatePytorchDataset(X_test_tensor, y_test_tensor)

In [14]:
# Creating a dataloader object:
test_loader = DataLoader(test_dataset, batch_size=512, shuffle=True)

# Loading and Testing the Best Model:

In [15]:
# To load the saved best model:
mlp_best_model = pickle.load(open('Models/mlp_sampled_model.sav', 'rb'))

In [16]:
# Choosing Cross Entropy Loss which combines logsoftmax and NLLLoss for use in multi-class classification:
criterion = nn.CrossEntropyLoss()

In [17]:
start = time.time()

# Setting the cumulative loss and accuracy to zero, to be updated within for loop below.
cum_test_loss = 0
cum_test_acc = 0

# Creating lists that will hold all predicted and actual target values tested
all_preds_list = []
all_actual_list = []

    
# TESTING #
mlp_best_model.eval()

for data, target in test_loader:
    # Predictions on test set batches
    y_pred_batch = mlp_best_model(data)

    # Cross-Entropy Loss and Accuracy:
    test_batch_loss = criterion(y_pred_batch, target.flatten())
    test_batch_acc = calc_acc_mlp(y_pred_batch, target.flatten())

    # Updating cumulative running totals:
    cum_test_loss += test_batch_loss.item()
    cum_test_acc += test_batch_acc.item()

    # Adding predictions/actual target to lists for confusion matrix:
    # Applies a softmax followed by a log to the y_pred tensor:
    y_pred_softmax = torch.log_softmax(y_pred_batch, dim = 1)
    # Returns the predicted values for each batch datapoint (the maximums within each logsoftmax'd tensor)
    _, y_pred_values = torch.max(y_pred_softmax, dim = 1) 

    # Converting predictions and actual target and storing in lists above:
    all_preds_list += (list(y_pred_values.numpy()))
    all_actual_list += (list(target.flatten().numpy()))

# PRINTING STATISTICS #
avg_test_loss = cum_test_loss / len(test_loader)
avg_test_acc = cum_test_acc / len(test_loader)


# Display:
print('Test Loss: {:.3f} | Test Acc: {:.3f}'.format(avg_test_loss, avg_test_acc))
        
print("Time taken (seconds): ", time.time() - start)  

Test Loss: 0.180 | Test Acc: 93.506
Time taken (seconds):  1.9268395900726318


In [27]:
print(classification_report(all_actual_list, all_preds_list, labels=[0, 1, 2, 3, 4, 5, 6]))

              precision    recall  f1-score   support

           0       0.93      0.94      0.93     42368
           1       0.96      0.93      0.94     56661
           2       0.94      0.96      0.95      7151
           3       0.85      0.91      0.88       549
           4       0.68      0.96      0.80      1899
           5       0.89      0.95      0.92      3473
           6       0.90      0.98      0.94      4102

    accuracy                           0.94    116203
   macro avg       0.88      0.95      0.91    116203
weighted avg       0.94      0.94      0.94    116203

